First, import needed modules

In [249]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point, LineString, Polygon
from geopy.geocoders import Nominatim
import os

Initalize variables

In [250]:
path = os.getcwd()
path

'/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters'

Load Dataset which has the endpoints of power line projects

In [251]:
powerlines = pd.read_csv(f'{path}/data/geodata/powerlines.csv', sep=';')
#powerlines = powerlines.set_index("# BBPlG 2013", drop = False)
powerlines.head()

,# BBPlG 2013,Start,End,New,Retro
0,1,Emden-Borssum,Osterath,NaN,NaN
1,2,Osterath,Philippsburg,NaN,NaN
2,3,Brunsbüttel,Großgartach,1.0,NaN
3,4,Wilster,Grafenrheinfeld,1.0,NaN
4,5,Bad Lauchstädt,Meitingen,1.0,NaN


Get the geocoordinates of the 'endpoint' cities using OSM/Nominatim API

In [252]:
start_locations = geocode(powerlines['Start'], provider='nominatim', user_agent='plm', timeout=4)
start_locations

,geometry,address
0,POINT (7.22429 53.34982),"Emden-Borssum, Birkenweg, Borssum, Emden, Nied..."
1,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh..."
2,POINT (9.13954 53.89725),"Brunsbüttel, Dithmarschen, Schleswig-Holstein,..."
3,POINT (9.37338 53.92328),"Wilster, Kreis Steinburg, Schleswig-Holstein, ..."
4,POINT (11.86870 51.38813),"Bad Lauchstädt, Lindenstraße, Lindenstraße, Kl..."
5,POINT (7.92070 51.68926),"Hamm-Uentrop, Geithenkamp, Geithe, Hamm, Nordr..."
6,POINT (13.93302 53.24422),"Bertikow, Uckerfelde, Gramzow, Uckermark, Bran..."
7,POINT (12.40904 51.15021),"Neukieritzsch, Landkreis Leipzig, Sachsen, 045..."
8,POINT (12.84531 50.86359),"Röhrsdorf, Chemnitz, Sachsen, 09247, Deutschland"
9,POINT (7.55674 50.36598),"Metternich, Koblenz, Rheinland-Pfalz, Deutschland"


In [253]:
end_locations = geocode(powerlines['End'], provider='nominatim', user_agent='plm', timeout=4)
end_locations

,geometry,address
0,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh..."
1,POINT (8.45467 49.23675),"Philippsburg, Verwaltungsverband Philippsburg,..."
2,POINT (9.12587 49.14254),"Großgartach, Leingarten, Landkreis Heilbronn, ..."
3,POINT (10.19668 50.00553),"Grafenrheinfeld, Landkreis Schweinfurt, Bayern..."
4,POINT (10.85000 48.55000),"Meitingen, Landkreis Augsburg, Bayern, 86405, ..."
5,POINT (7.41540 51.45458),"Kruckel, Hombruch, Dortmund, Nordrhein-Westfal..."
6,POINT (13.98890 53.50537),"Pasewalk, Vorpommern-Greifswald, Mecklenburg-V..."
7,POINT (11.14691 50.99545),"Vieselbach, Erfurt, Thüringen, Deutschland"
8,POINT (11.65955 50.52904),"Remptendorf, Saale-Orla-Kreis, Thüringen, 0736..."
9,POINT (6.49862 49.93028),"Niederstedem, Bitburger Land, Eifelkreis Bitbu..."


Merge into powerlines dataset

In [254]:
powerlines_geo_start = start_locations.join(powerlines, how = 'left')
powerlines_geo_start.rename(columns = {'geometry':'Start_geometry', 'address':'Start_address'}, inplace = True)
powerlines_geo = end_locations.join(powerlines_geo_start, how = 'left')
powerlines_geo.rename(columns = {'geometry':'End_geometry', 'address':'End_address'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['# BBPlG 2013', 'Start', 'Start_address', 'Start_geometry', 'End', 'End_address', 'End_geometry', 'direct_line', 'New', 'Retro'])
powerlines_geo

,# BBPlG 2013,Start,Start_address,Start_geometry,End,End_address,End_geometry,direct_line,New,Retro
0,1,Emden-Borssum,"Emden-Borssum, Birkenweg, Borssum, Emden, Nied...",POINT (7.22429 53.34982),Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),NaN,NaN,NaN
1,2,Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),Philippsburg,"Philippsburg, Verwaltungsverband Philippsburg,...",POINT (8.45467 49.23675),NaN,NaN,NaN
2,3,Brunsbüttel,"Brunsbüttel, Dithmarschen, Schleswig-Holstein,...",POINT (9.13954 53.89725),Großgartach,"Großgartach, Leingarten, Landkreis Heilbronn, ...",POINT (9.12587 49.14254),NaN,1.0,NaN
3,4,Wilster,"Wilster, Kreis Steinburg, Schleswig-Holstein, ...",POINT (9.37338 53.92328),Grafenrheinfeld,"Grafenrheinfeld, Landkreis Schweinfurt, Bayern...",POINT (10.19668 50.00553),NaN,1.0,NaN
4,5,Bad Lauchstädt,"Bad Lauchstädt, Lindenstraße, Lindenstraße, Kl...",POINT (11.86870 51.38813),Meitingen,"Meitingen, Landkreis Augsburg, Bayern, 86405, ...",POINT (10.85000 48.55000),NaN,1.0,NaN
5,9,Hamm-Uentrop,"Hamm-Uentrop, Geithenkamp, Geithe, Hamm, Nordr...",POINT (7.92070 51.68926),Kruckel,"Kruckel, Hombruch, Dortmund, Nordrhein-Westfal...",POINT (7.41540 51.45458),NaN,NaN,NaN
6,11,Bertikow,"Bertikow, Uckerfelde, Gramzow, Uckermark, Bran...",POINT (13.93302 53.24422),Pasewalk,"Pasewalk, Vorpommern-Greifswald, Mecklenburg-V...",POINT (13.98890 53.50537),NaN,NaN,NaN
7,13,Neukieritzsch,"Neukieritzsch, Landkreis Leipzig, Sachsen, 045...",POINT (12.40904 51.15021),Vieselbach,"Vieselbach, Erfurt, Thüringen, Deutschland",POINT (11.14691 50.99545),NaN,NaN,NaN
8,14,Röhrsdorf,"Röhrsdorf, Chemnitz, Sachsen, 09247, Deutschland",POINT (12.84531 50.86359),Remptendorf,"Remptendorf, Saale-Orla-Kreis, Thüringen, 0736...",POINT (11.65955 50.52904),NaN,NaN,NaN
9,15,Metternich,"Metternich, Koblenz, Rheinland-Pfalz, Deutschland",POINT (7.55674 50.36598),Niederstedem,"Niederstedem, Bitburger Land, Eifelkreis Bitbu...",POINT (6.49862 49.93028),NaN,NaN,NaN


Create direct line from start and end coordinates

In [255]:
for i in range (0, len(powerlines_geo)):
    start = powerlines_geo['Start_geometry'].loc[powerlines_geo.index[i]]
    end = powerlines_geo['End_geometry'].loc[powerlines_geo.index[i]]
    powerlines_geo['direct_line'].loc[powerlines_geo.index[i]] = LineString([start, end])
powerlines_geo.head()

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_25057/2775024114.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powerlines_geo['direct_line'].loc[powerlines_geo.index[i]] = LineString([start, end])
/opt/miniconda3/envs/python310/lib/python3.10/site-packages/pandas/core/internals/blocks.py:930: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return self.coerce_to_target_dtype(value).setitem(indexer, value)
/opt/miniconda3/envs/python310/lib/python3.10/site-packages/pandas/core/internals/managers.py:304: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)


,# BBPlG 2013,Start,Start_address,Start_geometry,End,End_address,End_geometry,direct_line,New,Retro
0,1,Emden-Borssum,"Emden-Borssum, Birkenweg, Borssum, Emden, Nied...",POINT (7.22429 53.34982),Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),"LINESTRING (7.22428778316919 53.349817, 6.6207...",NaN,NaN
1,2,Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),Philippsburg,"Philippsburg, Verwaltungsverband Philippsburg,...",POINT (8.45467 49.23675),"LINESTRING (6.6207151 51.2690922, 8.4546679 49...",NaN,NaN
2,3,Brunsbüttel,"Brunsbüttel, Dithmarschen, Schleswig-Holstein,...",POINT (9.13954 53.89725),Großgartach,"Großgartach, Leingarten, Landkreis Heilbronn, ...",POINT (9.12587 49.14254),"LINESTRING (9.1395423 53.8972549, 9.1258701 49...",1.0,NaN
3,4,Wilster,"Wilster, Kreis Steinburg, Schleswig-Holstein, ...",POINT (9.37338 53.92328),Grafenrheinfeld,"Grafenrheinfeld, Landkreis Schweinfurt, Bayern...",POINT (10.19668 50.00553),"LINESTRING (9.373378722135374 53.9232825, 10.1...",1.0,NaN
4,5,Bad Lauchstädt,"Bad Lauchstädt, Lindenstraße, Lindenstraße, Kl...",POINT (11.86870 51.38813),Meitingen,"Meitingen, Landkreis Augsburg, Bayern, 86405, ...",POINT (10.85000 48.55000),"LINESTRING (11.8686979 51.3881334, 10.85 48.55)",1.0,NaN


Export shapefile

In [256]:
powerlines_geo.rename(columns = {'direct_line':'geometry'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['#BBPlG2013', 'Start', 'End', 'geometry', 'New', 'Retro'])
powerlines_geo.to_file(f'{path}/data/geodata/powerlines.shp')

/opt/miniconda3/envs/python310/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
